<a href="https://colab.research.google.com/github/shivupoojar/Outlier-Detection-for-Puhatu-Monitoring-/blob/main/puhatu_c1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyod


In [ ]:
!pwd
%cd /content/drive/My Drive/Colab Notebooks/Outlier-Detection-for-Puhatu-Monitoring-
%cd data/



/content/drive/My Drive/Colab Notebooks/Outlier-Detection-for-Puhatu-Monitoring-/data
/content/drive/My Drive/Colab Notebooks/Outlier-Detection-for-Puhatu-Monitoring-
/content/drive/My Drive/Colab Notebooks/Outlier-Detection-for-Puhatu-Monitoring-/data
puhatu_b1.csv    puhatu_b3.csv    puhatu_c2v3.csv  puhatu_l1v3.csv
puhatu_b1v3.csv  puhatu_b3v3.csv  puhatu_c3.csv
puhatu_b2.csv    puhatu_c1.csv    puhatu_c3v3.csv
puhatu_b2v3.csv  puhatu_c2.csv    puhatu_l1.csv


In [ ]:
import pandas as pd
import numpy as np 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import io
df=pd.read_csv('puhatu_c1.csv',parse_dates=['time'],usecols=['time','wat_level_mH2o','dev_id','batt','rssi','snr','air_Pressure_mH2o','air_Temp_float','dist','wat_Pressure_mH2o','wat_Temp_float'])

In [ ]:
df = df.apply(pd.to_numeric, errors='coerce')
df['time'] = df['time'].astype('datetime64[ns]')
df = df.drop(['dev_id'], axis=1)
df.set_index('time',inplace=True)
#df = df.dropna()

#df['month']= df.index.month
df['year']= df.index.year
df = df.loc[df['year'] == 2021]
df = df.head(2590)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder
import scipy.stats as stats
import plotly.express as px


import matplotlib.font_manager
from pyod.models.cblof import CBLOF
# Import models
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
#from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from sklearn.cluster import DBSCAN
from pyod.models.ocsvm import OCSVM
# reading the big mart sales training data
from sklearn.preprocessing import MinMaxScaler

from joblib import dump, load

from sklearn.metrics import classification_report,accuracy_score

In [ ]:
# min max scalling
scaler = MinMaxScaler(feature_range=(0, 1))
# df[['wat_Pressure_mH2o','air_Pressure_mH2o','air_Temp_float','wat_Temp_float','dist']] = scaler.fit_transform(df[['wat_Pressure_mH2o','air_Pressure_mH2o','air_Temp_float','wat_Temp_float','dist']])
df[['wat_Pressure_mH2o','air_Pressure_mH2o','air_Temp_float','wat_Temp_float','dist']] = scaler.fit_transform(df[['wat_Pressure_mH2o','air_Pressure_mH2o','air_Temp_float','wat_Temp_float','dist']])
# X = df[['wat_Pressure_mH2o','air_Pressure_mH2o','air_Temp_float','wat_Temp_float','dist']]

In [ ]:
# declare the outlier mechanisms
import warnings
warnings.filterwarnings('ignore')

random_state = np.random.RandomState(42)
outliers_fraction = 0.3
# Define seven outlier detection tools to be compared
classifiers = {
     #    'Angle-based Outlier Detector (ABOD)': ABOD(contamination=outliers_fraction),
     #    'Cluster-based Local Outlier Factor (CBLOF)':CBLOF(contamination=outliers_fraction,check_estimator=False, random_state=random_state),
        #'Feature Bagging':FeatureBagging(LOF(n_neighbors=35),contamination=outliers_fraction,check_estimator=False,random_state=random_state),
     #    'Histogram-base Outlier Detection (HBOS)': HBOS(contamination=outliers_fraction),
        'Isolation Forest': IForest(n_estimators=35,contamination=outliers_fraction,random_state=random_state),
     #    'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
      #  'Average KNN': KNN(method='mean',contamination=outliers_fraction),
        'One Class SVM': OCSVM(contamination=outliers_fraction)
          
}

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from sklearn.model_selection import train_test_split
from pyod.utils.data import evaluate_print


xx , yy = np.meshgrid(np.linspace(0,1 , 200), np.linspace(0, 1, 200))
import seaborn as sns
for i, (clf_name, clf) in enumerate(classifiers.items()):
    
    xtrain, xtest = np.array_split(X,2)
    print(xtrain.shape)
    print(xtest.shape)
 #   xtrain, xtest, labels_train, labels_test = train_test_split(df, df['batt'], test_size=0.50, random_state=42)  
    clf.fit(xtrain)
    xtest1=xtest
    
   # prediction of a datapoint category outlier or inlier
    prediction_labels_train = clf.predict(xtrain)
    prediction_labels = clf.predict(xtest)
    prediction_score_xtrain = clf.decision_function(xtrain)
    prediction_score_xtest = clf.decision_function(xtest)
  
    
    
    y_true = prediction_labels_train
    y_pred = prediction_labels
    
    n_inliers = len(y_pred) - np.count_nonzero(y_pred)
    n_outliers = np.count_nonzero(y_pred == 1) 

#     dfx = xtest1
    
#     dfx['outlier'] = y_pred.tolist()

    
# evaluate and print the results
    print("\nOn Training Data:")
    evaluate_print(clf_name, y_true, prediction_score_xtrain)
    print("\nOn Test Data:")
    evaluate_print(clf_name, y_pred, prediction_score_xtest)
   
    print('################################## START ######################')
    print('Accuracy Score: ', accuracy_score(y_true, y_pred),clf_name)
    print('OUTLIERS : ',n_outliers,'INLIERS : ',n_inliers, clf_name)
    
    print(confusion_matrix(y_true, y_pred))

    print(classification_report(y_true, y_pred))

    # evaluate and print the results
   # print("\nOn Training Data:")
   # evaluate_print(clf_name, y_true, prediction_score_xtrain)
   # print("\nOn Test Data:")
  #  evaluate_print(clf_name, y_pred, prediction_score_xtest)
    print(xtrain)
    print(xtest) 

    print('################################## END ######################')
    visualize(clf_name, xtrain, y_true, xtest, y_pred, prediction_score_xtrain,
          prediction_score_xtest, show_figure=True, save_figure=False)

(1295, 5)
(1295, 5)

On Training Data:
Isolation Forest ROC:1.0, precision @ rank n:1.0

On Test Data:
Isolation Forest ROC:1.0, precision @ rank n:1.0
################################## START ######################
Accuracy Score:  0.2942084942084942 Isolation Forest
OUTLIERS :  1283 INLIERS :  12 Isolation Forest
[[  2 904]
 [ 10 379]]
              precision    recall  f1-score   support

           0       0.17      0.00      0.00       906
           1       0.30      0.97      0.45       389

    accuracy                           0.29      1295
   macro avg       0.23      0.49      0.23      1295
weighted avg       0.21      0.29      0.14      1295

                     wat_Pressure_mH2o  ...      dist
time                                    ...          
2021-01-07 06:13:20           0.751616  ...  0.033253
2021-01-07 06:13:20           0.751687  ...  0.033513
2021-01-07 06:13:20           0.751760  ...  0.037046
2021-01-07 06:13:20           0.751797  ...  0.032353
2021-01-0

ValueError: ignored